# 🚀 Unity Catalog Functions for Conversational Commerce Agent

## 📋 Overview
This notebook contains advanced Unity Catalog functions designed to power a sophisticated conversational commerce search application. The functions handle customer validation, personalized product recommendations with comprehensive filtering, and semantic search capabilities.

## 🔧 Enhanced Function Descriptions

### 1. `validate_customer_by_name(input_customer_name)` ✅
**Purpose**: Validates customers by name using AI similarity matching
**Use Case**: Identity verification for conversational queries
**Features**: 
- Fuzzy name matching with confidence scores
- Returns customer details and email for confirmation
- Handles name variations and typos

### 2. `get_smart_product_recommendations(...)` 🎯 **MAIN FUNCTION**
**Purpose**: Advanced hybrid recommendation engine with comprehensive filtering
**Use Cases**: 
- *"show me all the outdoor gears ordered by Charles Martinez"*
- *"non-fiction books from penguin whose price is below $20"*
- *"nike products under $100"*
- *"apple electronics between $200 and $800"*

**Enhanced Parameters**:
- `input_customer_name`: Customer name (AI fuzzy matching)
- `input_product_description`: Product category/description (optional)
- `input_brand`: Brand filter with fuzzy matching (optional)
- `input_min_price`: Minimum price filter (optional)
- `input_max_price`: Maximum price filter (optional)

**Advanced Features**:
- ✅ **Hybrid Vector Search**: User embeddings + description embeddings
- ✅ **Multi-Factor Scoring**: 35% user similarity + 25% description + 15% brand + 15% interactions + 10% quality
- ✅ **Smart Filtering**: Brand fuzzy matching + price range filtering
- ✅ **Explainable AI**: Detailed reasoning for each recommendation
- ✅ **Personalization**: Based on customer's unique interaction patterns

### 3. `product_lookup_by_description(input_description)` 🔍
**Purpose**: General semantic product search without customer context
**Use Case**: Anonymous product discovery and search
**Features**: Uses vector search on product descriptions for semantic matching

### 4. `get_random_customer_info()` 🎲 **NEW**
**Purpose**: Randomly selects a customer from the database
**Use Cases**: 
- *Testing and demonstration purposes*
- *Random customer selection for sampling*
- *Getting customer details for development/testing scenarios*

**Features**:
- ✅ **Random Selection**: Uses RAND() function for true randomization
- ✅ **Complete Profile**: Returns full customer information including demographics
- ✅ **Purchase History**: Includes total orders and spending data
- ✅ **No Parameters**: Simple function call with no input required

### 5. `get_customer_interactions(input_customer_name, input_customer_email)` 📊 **EXACT SEARCH**
**Purpose**: Looks up customer interaction history with products using precise customer identification
**Use Cases**: 
- *"Show me Charles Martinez's recent purchases and interactions"*
- *"Look up interactions for customer email: john.doe@email.com"*
- *Understanding customer behavior patterns*
- *Analyzing customer engagement with specific products*
- *Customer service and support scenarios*

**Parameters**:
- `input_customer_name`: Customer name (optional if email provided) - uses exact matching (case insensitive)
- `input_customer_email`: Customer email (optional if name provided) - uses exact matching (case insensitive)

**Features**:
- ✅ **Exact Identification**: Lookup by exact name OR email OR both
- ✅ **Precise Matching**: No fuzzy matching - requires exact name/email match
- ✅ **Case Insensitive**: Both name and email matching work regardless of case
- ✅ **Smart Prioritization**: Email gets priority over name when both provided
- ✅ **Comprehensive History**: Returns up to 10 recent interactions per customer
- ✅ **Rich Details**: Includes product names, prices, categories, and brands
- ✅ **Interaction Types**: Views, cart additions, purchases, ratings, and reviews
- ✅ **Time Analytics**: Shows timestamps and days since interaction
- ✅ **High Precision**: Eliminates ambiguity with exact matching

## 🎯 Conversational Agent Workflow

### **Recommended 2-Step Process**:

1. **Customer Validation**:
   ```sql
   SELECT * FROM validate_customer_by_name("Charles Martinez");
   ```
   → *"I found Charles Martinez with email charles.martinez@email.com. Is this correct?"*

2. **Smart Recommendations** (handles everything):
   ```sql
   -- Simple query
   SELECT * FROM get_smart_product_recommendations("Charles Martinez", "outdoor gear", NULL, NULL, NULL);
   
   -- Complex query with filters
   SELECT * FROM get_smart_product_recommendations("Charles Martinez", "non-fiction books", "penguin", NULL, 20.0);
   ```

## 🔬 Technical Architecture

### **Vector Search Implementation**:
- **User Embeddings**: `product_index` with customer interaction history
- **Product Embeddings**: `product_desc_index` with semantic description matching
- **Hybrid Approach**: Combines both for optimal results

### **AI-Powered Features**:
- `ai_similarity()` for fuzzy matching (names, brands, categories)
- Vector embeddings for semantic understanding
- Multi-signal scoring algorithm for relevance ranking

### **Performance Optimizations**:
- Pre-computed embeddings in Unity Catalog vector indexes
- Efficient filtering before scoring
- Optimized for sub-second response times

## 🎨 Integration Benefits

- **Single Function**: One call handles personalization + filtering + search
- **Natural Language Ready**: Perfect for conversational AI agents
- **Explainable Results**: Each recommendation includes reasoning
- **Flexible Parameters**: Works with partial information
- **Scalable Architecture**: Uses Databricks vector search infrastructure
- **Rich Metadata**: Returns images, ratings, prices, and detailed product info

## 📊 Function Comparison

| Function | Primary Use | Approach | Personalization |
|----------|-------------|----------|-----------------|
| `get_smart_product_recommendations` | **MAIN** - Personalized search with filters | Hybrid Vector + AI Filtering | ✅ High |
| `validate_customer_by_name` | Customer validation | AI Similarity Matching | ❌ None |
| `product_lookup_by_description` | General product search | Description Vector Search | ❌ None |
| `get_random_customer_info` | **NEW** - Random customer selection | Random Database Sampling | ❌ None |
| `get_customer_interactions` | **EXACT SEARCH** - Customer behavior analysis | Exact Name/Email + History | ✅ Customer-Specific |

**💡 Recommendation**: Use `get_smart_product_recommendations` as your primary function - it's designed to handle complex conversational commerce queries with multiple constraints and personalization!


In [0]:
%sql
use catalog retail_consumer_goods;
use retail_consumer_goods.conversational_commerce_search;

In [0]:
%sql
-- Clean up existing functions
DROP FUNCTION IF EXISTS validate_customer_by_name;

DROP FUNCTION IF EXISTS get_smart_product_recommendations;

DROP FUNCTION IF EXISTS product_lookup_by_description;

DROP FUNCTION IF EXISTS get_random_customer_info;

DROP FUNCTION IF EXISTS get_customer_interactions;

In [0]:
%sql
-- 1. Customer Validation Function
-- This function validates a customer by name and returns their email for confirmation
CREATE OR REPLACE FUNCTION validate_customer_by_name(
    input_customer_name STRING COMMENT "Customer's name to validate"
)
RETURNS TABLE(
    customer_id STRING COMMENT "Customer's unique identifier",
    customer_name STRING COMMENT "Customer's full name",
    customer_email STRING COMMENT "Customer's email address for validation",
    name_similarity_score DOUBLE COMMENT "Name matching confidence score"
)
COMMENT "This function finds customers by name and returns their email for validation. Uses AI similarity matching to handle variations in name input."
LANGUAGE SQL
RETURN
    SELECT 
        customer_id,
        name as customer_name,
        email as customer_email,
        ai_similarity(name, input_customer_name) as name_similarity_score
    FROM retail_consumer_goods.conversational_commerce_search.customers
    WHERE ai_similarity(name, input_customer_name) > 0.8
    ORDER BY name_similarity_score DESC
    LIMIT 5;
    

In [0]:
%sql
CREATE OR REPLACE FUNCTION get_smart_product_recommendations(
    input_customer_name STRING COMMENT "Customer's name",
    input_customer_email STRING COMMENT "Customer's email for precise identification (recommended to avoid name conflicts)",
    input_product_description STRING COMMENT "Description of what the customer is looking for (e.g., 'outdoor camping gear', 'fiction books', 'wireless headphones')",
    input_brand STRING DEFAULT NULL COMMENT "Brand filter (e.g., 'penguin', 'nike', 'apple') - uses fuzzy matching",
    input_min_price DOUBLE DEFAULT NULL COMMENT "Minimum price filter (e.g., 10.0 for products above 10 $)",
    input_max_price DOUBLE DEFAULT NULL COMMENT "Maximum price filter (e.g., 20.0 for products under 20 $)"
)
RETURNS TABLE(
    customer_name STRING COMMENT "Customer's name",
    customer_email STRING COMMENT "Customer's email",
    product_id STRING COMMENT "Product's unique identifier",
    product_name STRING COMMENT "Product's name",
    category STRING COMMENT "Product's category",
    subcategory STRING COMMENT "Product's subcategory",
    brand STRING COMMENT "Product's brand",
    price DOUBLE COMMENT "Product's price",
    description STRING COMMENT "Product's description",
    avg_rating DOUBLE COMMENT "Product's average rating",
    recommendation_score DOUBLE COMMENT "Combined recommendation score (higher is better)",
    recommendation_reason STRING COMMENT "Explanation of why this product was recommended",
    brand_match_score DOUBLE COMMENT "Brand similarity score (if brand filter applied)",
    price_in_range BOOLEAN COMMENT "Whether product price meets the specified criteria"
)
COMMENT "Optimized smart recommendation function that conditionally skips expensive collaborative filtering when a specific brand is requested."
LANGUAGE SQL
RETURN
    WITH customer_context AS (
        SELECT 
            c.customer_id,
            c.name AS customer_name,
            c.email AS customer_email,
            uf.embedding embedding
        FROM retail_consumer_goods.conversational_commerce_search.customers c
        JOIN retail_consumer_goods.conversational_commerce_search.user_features uf
            ON c.customer_id = uf.customer_id
        WHERE LOWER(c.email) = LOWER(input_customer_email)
          AND LOWER(c.name) = LOWER(input_customer_name)
        LIMIT 1
    ),
    
    -- [OPTIMIZATION] GATEKEEPER CTE
    -- Only triggers personalization (History & Similar Users) if NO BRAND is specified.
    personalization_request AS (
        SELECT embedding, customer_id
        FROM customer_context
        WHERE input_brand IS NULL
    ),

    -- 1. Find similar users (Collaborative Filtering) - Skipped if brand is set
    -- [OPTIMIZATION] Scalar subquery is required for VECTOR_SEARCH - planner will optimize
    similar_users AS (
        SELECT 
            CAST(sp.customer_id AS STRING) AS similar_customer_id,
            sp.search_score AS similarity_score
        FROM personalization_request pr
        CROSS JOIN VECTOR_SEARCH(
            index => 'retail_consumer_goods.conversational_commerce_search.user_index',
            query_vector => (SELECT embedding FROM personalization_request LIMIT 1),
            num_results => 10
        ) sp
        WHERE sp.customer_id <> pr.customer_id
    ),
    
    -- 2. Get products liked by similar users - Skipped if similar_users is empty
    -- [OPTIMIZATION] Use INNER JOIN explicitly and remove nested subquery
    neighbor_candidates AS (
        SELECT 
            cpf.product_id,
            SUM(cpf.interaction_count * su.similarity_score) AS neighbor_signal,
            MAX(cpf.avg_customer_rating) AS neighbor_avg_rating
        FROM retail_consumer_goods.conversational_commerce_search.customer_product_interaction_features cpf
        INNER JOIN similar_users su
            ON cpf.customer_id = su.similar_customer_id
        GROUP BY cpf.product_id
        ORDER BY neighbor_signal DESC
        LIMIT 20
    ),

    -- 3. User History Recommendations - Skipped if brand is set
    -- [OPTIMIZATION] Scalar subquery is required for VECTOR_SEARCH - planner will optimize
    vector_recommendations AS (
        SELECT 
            sp.product_id,
            sp.search_score AS vector_score
        FROM personalization_request pr
        CROSS JOIN VECTOR_SEARCH(
            index => 'retail_consumer_goods.conversational_commerce_search.product_index',
            query_vector => (SELECT embedding FROM personalization_request LIMIT 1),
            num_results => 50
        ) sp
        WHERE 
             (input_min_price IS NULL OR sp.price >= input_min_price)
          AND (input_max_price IS NULL OR sp.price <= input_max_price)
    ),

    -- 4. Description Search (Semantic Search) - ALWAYS runs (Primary Intent)
    description_recommendations AS (
        SELECT 
            sp.product_id,
            sp.search_score AS description_score
        FROM VECTOR_SEARCH(
            index => 'retail_consumer_goods.conversational_commerce_search.product_desc_index',
            query_text => input_product_description,
            num_results => 50
        ) sp
        WHERE 
             (input_min_price IS NULL OR sp.price >= input_min_price)
          AND (input_max_price IS NULL OR sp.price <= input_max_price)
          AND (
              input_brand IS NULL 
              OR LOWER(sp.brand) = LOWER(input_brand)
              OR CONTAINS(LOWER(sp.brand), LOWER(input_brand))
              OR (1.0 - (CAST(levenshtein(LOWER(sp.brand), LOWER(input_brand)) AS DOUBLE) / GREATEST(LENGTH(sp.brand), LENGTH(input_brand), 1))) > 0.6
          )
    ),

    -- [OPTIMIZATION] Use UNION ALL with 0.0 instead of NULL for faster MAX aggregation
    -- MAX on 0.0 values is much faster than MAX on NULL values
    candidate_products AS (
        SELECT product_id, vector_score, 0.0 AS description_score, 0.0 AS neighbor_signal, 0.0 AS neighbor_avg_rating
        FROM vector_recommendations
        UNION ALL
        SELECT product_id, 0.0, description_score, 0.0, 0.0
        FROM description_recommendations
        UNION ALL
        SELECT product_id, 0.0, 0.0, neighbor_signal, neighbor_avg_rating
        FROM neighbor_candidates
    ),

    merged_candidates AS (
        SELECT 
            product_id,
            MAX(vector_score) AS vector_score,
            MAX(description_score) AS description_score,
            MAX(neighbor_signal) AS neighbor_signal,
            MAX(neighbor_avg_rating) AS neighbor_avg_rating
        FROM candidate_products
        GROUP BY product_id
        -- Filter out products with all zero scores (not recommended by any source)
        -- Note: neighbor_avg_rating can be > 0 even if neighbor_signal is 0
        HAVING MAX(vector_score) > 0.0 
            OR MAX(description_score) > 0.0 
            OR MAX(neighbor_signal) > 0.0
            OR MAX(neighbor_avg_rating) > 0.0
    ),

    product_features AS (
        SELECT 
            mc.product_id,
            p.name AS product_name,
            p.category,
            p.subcategory,
            p.brand,
            p.price,
            p.description,
            p.avg_rating,
            COALESCE(mc.vector_score, 0.0) AS vector_score,
            COALESCE(mc.description_score, 0.0) AS description_score,
            COALESCE(mc.neighbor_signal, 0.0) AS neighbor_signal,
            COALESCE(mc.neighbor_avg_rating, 0.0) AS neighbor_avg_rating,
            CASE 
                WHEN input_brand IS NULL THEN 1.0
                WHEN LOWER(p.brand) = LOWER(input_brand) THEN 1.0
                WHEN contains(LOWER(p.brand), LOWER(input_brand)) THEN 0.9
                ELSE 
                    GREATEST(0.0, 1.0 - (
                        CAST(levenshtein(LOWER(p.brand), LOWER(input_brand)) AS DOUBLE) / 
                        GREATEST(LENGTH(p.brand), LENGTH(input_brand), 1)
                    ))
            END AS brand_match_score,
            CASE 
                WHEN input_min_price IS NOT NULL AND input_max_price IS NOT NULL THEN (p.price BETWEEN input_min_price AND input_max_price)
                WHEN input_min_price IS NOT NULL THEN (p.price >= input_min_price)
                WHEN input_max_price IS NOT NULL THEN (p.price <= input_max_price)
                ELSE TRUE
            END AS price_in_range
        FROM merged_candidates mc
        INNER JOIN retail_consumer_goods.conversational_commerce_search.products p
            ON mc.product_id = p.product_id
        WHERE p.in_stock = TRUE
    ),

    filtered_products AS (
        SELECT *
        FROM product_features
        WHERE price_in_range
          AND (input_brand IS NULL OR brand_match_score > 0.6)
    ),

    scored_recommendations AS (
        SELECT
            cc.customer_name,
            cc.customer_email,
            fp.product_id,
            fp.product_name,
            fp.category,
            fp.subcategory,
            fp.brand,
            fp.price,
            fp.description,
            fp.avg_rating,
            fp.brand_match_score,
            fp.price_in_range,
            fp.vector_score,
            fp.description_score,
            fp.neighbor_signal,
            fp.neighbor_avg_rating,
            COALESCE(cif.interaction_count, 0) AS interaction_count,
            COALESCE(cif.avg_customer_rating, 0) AS customer_rating,
            
            -- [OPTIMIZATION] Dynamic Scoring Weights based on Intent
            (
                COALESCE(fp.vector_score, 0) * (CASE WHEN input_brand IS NULL THEN 0.30 ELSE 0.10 END) +
                COALESCE(fp.description_score, 0) * (CASE WHEN input_brand IS NULL THEN 0.25 ELSE 0.55 END) +
                CASE WHEN COALESCE(cif.interaction_count, 0) > 0 THEN 0.15 ELSE 0.10 END +
                (fp.avg_rating / 5.0) * 0.10 +
                CASE WHEN input_brand IS NOT NULL THEN fp.brand_match_score * 0.15 ELSE 0 END +
                (LEAST(COALESCE(fp.neighbor_signal, 0), 25) / 25.0) * 0.10
            ) AS recommendation_score,

            CASE 
                WHEN COALESCE(cif.interaction_count, 0) > 0 THEN CONCAT('Based on your previous interactions and match for "', input_product_description, '"')
                WHEN input_brand IS NOT NULL AND fp.brand_match_score > 0.8 THEN CONCAT('Top pick from ', input_brand, ' matching "', input_product_description, '"')
                WHEN COALESCE(fp.neighbor_signal, 0) > 0 THEN 'Popular among customers with similar profiles'
                WHEN COALESCE(fp.description_score, 0) > 0.7 THEN CONCAT('Best match for "', input_product_description, '"')
                ELSE 'Recommended based on your preferences'
            END AS recommendation_reason

        FROM customer_context cc
        CROSS JOIN filtered_products fp
        LEFT JOIN retail_consumer_goods.conversational_commerce_search.customer_product_interaction_features cif
            ON cc.customer_id = cif.customer_id
           AND fp.product_id = cif.product_id
    )

    SELECT 
        customer_name,
        customer_email,
        product_id,
        product_name,
        category,
        subcategory,
        brand,
        price,
        description,
        avg_rating,
        recommendation_score,
        recommendation_reason,
        brand_match_score,
        price_in_range
    FROM scored_recommendations
    ORDER BY recommendation_score DESC
    LIMIT 25;     


In [0]:
%sql
-- ENHANCED: Product Lookup with Exact Name Match Priority
-- First tries exact name matching, then falls back to vector similarity search
-- 
-- USAGE: Replace the existing product_lookup_by_description function in 03_uc_function_tools.ipynb
-- with this enhanced version
--
-- IMPROVEMENTS:
-- 1. ✅ Tries exact product name matching first (case-insensitive)
-- 2. ✅ Falls back to vector similarity search if no exact match
-- 3. ✅ Now returns brand and avg_rating fields (fixes the "Unknown" brand issue)
-- 4. ✅ Returns search_score = 1.0 for exact matches (perfect score)
-- 5. ✅ Maintains semantic search capability for fuzzy queries

CREATE OR REPLACE FUNCTION product_lookup_by_description(
    input_description STRING COMMENT "User Input for product description or exact product name"
)
RETURNS TABLE(
    product_id STRING COMMENT "Product's unique identifier",
    name STRING COMMENT "Product's name",
    category STRING COMMENT "Product's category",
    subcategory STRING COMMENT "Product's subcategory",
    brand STRING COMMENT "Product's brand",
    price DOUBLE COMMENT "Product's price",
    description STRING COMMENT "Product's description",
    avg_rating DOUBLE COMMENT "Product's average rating",
    search_score DOUBLE COMMENT "Product's search score (1.0 for exact match, vector score otherwise)"  
)
COMMENT "This function returns product details for a given user description. PRIORITY: First checks for exact product name match (case-insensitive), then falls back to semantic vector search if no exact match found."
LANGUAGE SQL
RETURN
WITH exact_name_match AS (
    -- Try exact name matching first (case-insensitive)
    SELECT 
        p.product_id,
        p.name,
        p.category,
        p.subcategory,
        p.brand,
        p.price,
        p.description,
        p.avg_rating,
        1.0 as search_score  -- Perfect score for exact match
    FROM retail_consumer_goods.conversational_commerce_search.products p
    WHERE LOWER(p.name) = LOWER(input_description)
    LIMIT 5
),
vector_search_results AS (
    -- Fallback to vector similarity search if no exact match
    SELECT 
        sp.product_id,
        p.name,
        p.category,
        p.subcategory,
        p.brand,
        p.price,
        p.description,
        p.avg_rating,
        sp.search_score
    FROM VECTOR_SEARCH(
        index => 'retail_consumer_goods.conversational_commerce_search.product_desc_index',
        query_text => input_description,
        num_results => 5
    ) sp
    JOIN retail_consumer_goods.conversational_commerce_search.products p 
        ON sp.product_id = p.product_id 
)
-- Return exact matches if found, otherwise return vector search results
SELECT * FROM exact_name_match
WHERE EXISTS (SELECT 1 FROM exact_name_match)
UNION ALL
SELECT * FROM vector_search_results
WHERE NOT EXISTS (SELECT 1 FROM exact_name_match)
ORDER BY search_score DESC;



In [0]:
%sql
-- 4. Random Customer Lookup Function
-- This function randomly selects a customer from the customers table and returns their name and email
CREATE OR REPLACE FUNCTION get_random_customer_info()
RETURNS TABLE(
    customer_id STRING COMMENT "Customer's unique identifier",
    customer_name STRING COMMENT "Customer's full name",
    customer_email STRING COMMENT "Customer's email address",
    customer_age INT COMMENT "Customer's age",
    customer_gender STRING COMMENT "Customer's gender",
    customer_location STRING COMMENT "Customer's location",
    persona STRING COMMENT "Customer's persona/profile type",
    preferred_categories STRING COMMENT "Customer's preferred product categories",
    total_orders INT COMMENT "Total number of orders placed by customer",
    total_spent DOUBLE COMMENT "Total amount spent by customer"
)
COMMENT "This function randomly selects a customer from the customers table and returns their complete profile information including name, email, demographics, and purchase history. Useful for testing, demos, or random customer selection scenarios."
LANGUAGE SQL
RETURN
    SELECT 
        customer_id,
        name as customer_name,
        email as customer_email,
        age as customer_age,
        gender as customer_gender,
        location as customer_location,
        persona,
        preferred_categories,
        total_orders,
        total_spent
    FROM retail_consumer_goods.conversational_commerce_search.customers
    ORDER BY RAND()  -- Random ordering
    LIMIT 1;  -- Return only one random customer


In [0]:
%sql
-- 5. Customer Interactions Lookup Function (Enhanced with Exact Search)
-- This function looks up customer interactions with products using exact name or email matching
CREATE OR REPLACE FUNCTION get_customer_interactions(
    input_customer_name STRING DEFAULT NULL COMMENT "Customer's name to look up interactions for (optional if email provided)",
    input_customer_email STRING DEFAULT NULL COMMENT "Customer's email to look up interactions for (optional if name provided)"
)
RETURNS TABLE(
    customer_name STRING COMMENT "Customer's full name",
    customer_email STRING COMMENT "Customer's email address",
    product_name STRING COMMENT "Product's name",
    product_price DOUBLE COMMENT "Product's price",
    interaction_type STRING COMMENT "Type of interaction (view, cart_add, purchase, rating, review)",
    interaction_timestamp TIMESTAMP COMMENT "When the interaction occurred",
    customer_rating DOUBLE COMMENT "Rating given by customer (if applicable)",
    product_category STRING COMMENT "Product's category",
    product_brand STRING COMMENT "Product's brand",
    days_since_interaction INT COMMENT "Number of days since the interaction occurred"
)
COMMENT "This function looks up a customer's interaction history with products using exact name or email matching, returning up to 10 different interactions including customer name, product details, prices, and interaction types. Supports precise customer lookup by exact name match (case insensitive) or exact email match."
LANGUAGE SQL
RETURN
    WITH customer_match AS (
        SELECT customer_id, name, email
        FROM retail_consumer_goods.conversational_commerce_search.customers
        WHERE 
            -- Match by email (exact match, highest priority)
            (input_customer_email IS NOT NULL AND LOWER(email) = LOWER(input_customer_email))
            OR
            -- Match by name (exact match, case insensitive)
            (input_customer_name IS NOT NULL AND LOWER(name) = LOWER(input_customer_name))
        ORDER BY 
            -- Prioritize exact email matches, then exact name matches
            CASE WHEN input_customer_email IS NOT NULL AND LOWER(email) = LOWER(input_customer_email) THEN 1 ELSE 2 END
        LIMIT 1
    )
    SELECT 
        cm.name as customer_name,
        cm.email as customer_email,
        p.name as product_name,
        p.price as product_price,
        i.interaction_type,
        i.timestamp as interaction_timestamp,
        i.rating as customer_rating,
        p.category as product_category,
        p.brand as product_brand,
        DATEDIFF(CURRENT_DATE(), DATE(i.timestamp)) as days_since_interaction
    FROM customer_match cm
    JOIN retail_consumer_goods.conversational_commerce_search.interactions i 
        ON cm.customer_id = i.customer_id
    JOIN retail_consumer_goods.conversational_commerce_search.products p 
        ON i.product_id = p.product_id
    ORDER BY i.timestamp DESC  -- Most recent interactions first
    LIMIT 10;
